In [20]:
%load_ext autoreload
%autoreload 2

In [21]:
import sys
sys.path.append('../')

import shutil
from train import run
import os
import itertools
import glob

def write_errormsg2file(msg, error_file_name):
    if not os.path.isfile(error_file_name):
        with open(error_file_name, 'w') as f:
            f.write(f'error : {msg}\n')
    else:
        with open(error_file_name, 'a') as f:
            f.write(f'error : {msg}\n')
            
def do_exps(exps_dict= None, general_opts= None, device= None, exp_dir = '../figs/test', save_special= False):    
    exp_idx = 0
    keys= list(exps.keys())
    
    val_list_list= []
    key_list = [] #eg: 'MODEL.MODEL_A.rotation_lambda'
    key_suffix_list= [] # eg: 'rotation_lambda'
    
    for key, val_list in exps_dict.items():
        key_list.append(key)
        key_suffix_list.append(key.split('.')[-1])
        
        val_list_list.append(val_list)
        
    attr_combination_list = [list(s) for s in itertools.product(*val_list_list)]
    
    print(f'number of total experiments : {len(attr_combination_list)}')
    
    count_already_trained=0
    count_train_from_begining=0
    for attr_combination in attr_combination_list:
        save_dir = f'{exp_dir}/'
        opts= []
        for idx in range(len(attr_combination)):
            opts += [key_list[idx], attr_combination[idx]]
            attr= attr_combination[idx]
            
            #####
            attr_is_list= False
            try:attr_is_list = isinstance(eval(attr), list)
            except:pass
            if attr_is_list:
                attr= '_'.join(list(map(str, eval(attr)))) ## [, ] should not be in the directory name because glob is sensitive to that !
            #####
            
            save_dir+= f'{key_suffix_list[idx]}({attr})@'
    
        save_dir = save_dir[:-1] # remove last '@' 

        exp_idx+=1
        opts_other= ['NAME', f'exp_idx({exp_idx})', 
                     'GENERAL.device', device, 
                     'GENERAL.save_dir', save_dir
                    ]
                     
        opts_other+= general_opts

        opts = opts_other + opts

        
        if len(glob.glob(f'{save_dir}/1_*.jpg'))!=0:
            count_already_trained+=1
            print(f'PASSING :: {save_dir}')
            continue
        
        count_train_from_begining+=1
                    
        try:shutil.rmtree(save_dir)
        except:pass

        save_folder_name= save_dir.split('/')[-1]
        if len(save_folder_name)>255:
            print(f'\nFolder length is too long: len(results_saving_folder) -> {len(save_folder_name)} (<= 255)')
            print(save_folder_name)
        
        run(opts= opts, save_special=save_special)
        #try:
        #    run(opts= opts, save_special=save_special)
        #except Exception as e:
        #    error_file_name = f'{exp_dir}/errors.txt'
        #    write_errormsg2file(f'ERROR : {save_dir}\n {e} \n\n', error_file_name)
        #    print(f'ERROR : {save_dir}\n {e} \n\n')
    print('count_already_trained : ', count_already_trained)
    print('count_train_from_begining : ', count_train_from_begining)

In [22]:
exp_dir= '/n/holyscratch01/wadduwage_lab/uom_Udith/results/aim2/important/exp_set1'
#!rm -rf $exp_dir
#!mkdir $exp_dir

In [23]:
'''
device = 'cuda:0' # estimated: 40 hours !!! [9:34 AM, 2021/10/24]

exps = {
        'DATASET.name': ['confocal', 'mnistdigits_grid2patch'], 
    
        'MODEL.MODEL_A.rotation_lambda': ['10.0', '10000.0'],
        'MODEL.MODEL_A.lambda_scale_factor': ['1','2','3','4', '5', '6', '7', '8'],  
        'MODEL.MODEL_H.T': ['32'], #, '16', '8', '4', '2', '1'
        'MODEL.MODEL_H.lr_H': ['1.0', '0.0'], # '0.0'
        'MODEL.MODEL_H.H_init': ['randn_FourierBased'],
    
        
        'DATASET.img_size':  ['256'],
        'DATASET.num_samples_train': ['3000'],
        'MODEL.MODEL_DECODER.upsample_net': ['learnable_transpose_conv']} #'custom_v2' -> check what is better !!!

general_opts= ['TRAIN.show_results_epoch', '5',
                'TRAIN.epochs', '150']

do_exps(exps, general_opts, device, exp_dir = exp_dir, save_special= True)
'''

"\ndevice = 'cuda:0' # estimated: 40 hours !!! [9:34 AM, 2021/10/24]\n\nexps = {\n        'DATASET.name': ['confocal', 'mnistdigits_grid2patch'], \n    \n        'MODEL.MODEL_A.rotation_lambda': ['10.0', '10000.0'],\n        'MODEL.MODEL_A.lambda_scale_factor': ['1','2','3','4', '5', '6', '7', '8'],  \n        'MODEL.MODEL_H.T': ['32'], #, '16', '8', '4', '2', '1'\n        'MODEL.MODEL_H.lr_H': ['1.0', '0.0'], # '0.0'\n        'MODEL.MODEL_H.H_init': ['randn_FourierBased'],\n    \n        \n        'DATASET.img_size':  ['256'],\n        'DATASET.num_samples_train': ['3000'],\n        'MODEL.MODEL_DECODER.upsample_net': ['learnable_transpose_conv']} #'custom_v2' -> check what is better !!!\n\ngeneral_opts= ['TRAIN.show_results_epoch', '5',\n                'TRAIN.epochs', '150']\n\ndo_exps(exps, general_opts, device, exp_dir = exp_dir, save_special= True)\n"

In [ ]:
device = 'cuda:0' # estimated: 40 hours !!! [9:34 AM, 2021/10/24]

## remain:
#1. confocal, 10.0, lambda_scale_factor= 4, lr_H= 0.0


exps = {
        'DATASET.name': ['confocal', 'mnistdigits_grid2patch'], 
    
        'MODEL.MODEL_A.rotation_lambda': ['10.0', '10000.0'],
        'MODEL.MODEL_A.lambda_scale_factor': ['5', '6', '7', '8'],  
        'MODEL.MODEL_H.T': ['32'], #, '16', '8', '4', '2', '1'
        'MODEL.MODEL_H.lr_H': ['1.0', '0.0'], # '0.0'
        'MODEL.MODEL_H.H_init': ['randn_FourierBased'],
    
        
        'DATASET.img_size':  ['256'],
        'DATASET.num_samples_train': ['3000'],
        'MODEL.MODEL_DECODER.upsample_net': ['learnable_transpose_conv']} #'custom_v2' -> check what is better !!!

general_opts= ['TRAIN.show_results_epoch', '5',
                'TRAIN.epochs', '150']

do_exps(exps, general_opts, device, exp_dir = exp_dir, save_special= True)

In [19]:
import glob

glob.glob(f'{exp_dir}/*lambda_scale_factor(4)*T(32)*lr_H(0.0)*H_init(randn_FourierBased)*/*')

['/n/holyscratch01/wadduwage_lab/uom_Udith/results/aim2/important/exp_set1/name(confocal)@rotation_lambda(10.0)@lambda_scale_factor(4)@T(32)@lr_H(0.0)@H_init(randn_FourierBased)@img_size(256)@num_samples_train(3000)@upsample_net(learnable_transpose_conv)/40_L1Loss(0.0359553)_MSE(0.0045335)_SSIM11(0.6211371)_SSIM5(0.6217381).jpg',
 '/n/holyscratch01/wadduwage_lab/uom_Udith/results/aim2/important/exp_set1/name(confocal)@rotation_lambda(10.0)@lambda_scale_factor(4)@T(32)@lr_H(0.0)@H_init(randn_FourierBased)@img_size(256)@num_samples_train(3000)@upsample_net(learnable_transpose_conv)/100_L1Loss(0.0343714)_MSE(0.0038551)_SSIM11(0.6304208)_SSIM5(0.6306141).jpg',
 '/n/holyscratch01/wadduwage_lab/uom_Udith/results/aim2/important/exp_set1/name(confocal)@rotation_lambda(10.0)@lambda_scale_factor(4)@T(32)@lr_H(0.0)@H_init(randn_FourierBased)@img_size(256)@num_samples_train(3000)@upsample_net(learnable_transpose_conv)/5_L1Loss(0.0378053)_MSE(0.0055266)_SSIM11(0.6095156)_SSIM5(0.6107824).jpg',
 '/n